In [758]:
import torch
import time
import torch.autograd.functional as F

In [759]:
    n = torch.normal(0, 1, size=(100, 1))

In [760]:
def func(x, p):
    x = x.unsqueeze(0)
    i = torch.ones(x.shape)
    x_2 = x**2
    X = torch.cat((x_2, x, i), dim = 0).T
    y = torch.exp(torch.matmul(X, p)) + n
    return X, y


In [761]:
def error(x, p, p_):
    _, y = func(x, p)
    _, y_ = func(x, p_)
    return y - y_

In [762]:
x = torch.arange(100) / 100
p = torch.tensor([1, 2, 1]).float().unsqueeze(1)
p_ = torch.tensor([1, -2, 5]).float().unsqueeze(1)
X, y = func(x, p)

In [763]:
iterations = 100
lr = 0.5
e_sum = 0
u = 1
c = 0
start_time = time.time()
for i in range(iterations):
    e = error(x, p, p_)
    e_sum = torch.sum(e)
    J = F.jacobian(lambda p_: error(x, p, p_), p_)
    J = J.reshape(100, 3)
    H = J.T @ J
    H_ = H + u * torch.eye(H.shape[0])
    g = -J.T @ e
    dp = torch.inverse(H_) @ g
    _, z = func(x, p_)
    if c < 0.41:
        u *= 2
    elif c > 0.45:
        u /= 3
    p_ += dp * lr  
    _, z_ = func(x, p_)
    print(f'cost: {e_sum}')
    c = torch.norm(z - z_) / torch.norm(J @ dp)
    if(dp.norm() < 1e-6):
        print(i)
        break
    
end_time = time.time()
print(f'result: {p_}')
print(f'duration: {end_time - start_time}')


cost: -6559.72802734375
cost: -3902.5634765625
cost: -2305.87841796875
cost: -1349.213134765625
cost: -779.3309936523438
cost: -443.4924011230469
cost: -248.59352111816406
cost: -137.86178588867188
cost: -76.6369857788086
cost: -43.481998443603516
cost: -24.093385696411133
cost: -12.863399505615234
cost: -6.6605224609375
cost: -3.3870837688446045
cost: -1.7072129249572754
cost: -0.856957197189331
cost: -0.4293184280395508
cost: -0.21483945846557617
cost: -0.10738849639892578
cost: -0.05363893508911133
cost: -0.02668142318725586
cost: -0.013309001922607422
cost: -0.006612300872802734
cost: -0.0033173561096191406
cost: -0.001522064208984375
cost: -0.0006873607635498047
cost: -0.00030922889709472656
cost: -0.00013208389282226562
cost: -0.00013113021850585938
cost: -0.000110626220703125
cost: -6.222724914550781e-05
cost: -8.559226989746094e-05
31
result: tensor([[1.0000],
        [2.0000],
        [1.0000]])
duration: 0.0862877368927002
